In [1]:
import torch
from torch.utils.data import Dataset
import glob
import os
import numpy as np
import pickle
from PIL import Image
from tqdm import tqdm
import cv2

In [2]:
def save_dict(di_, filename_):
    with open(filename_, 'wb') as f:
        pickle.dump(di_, f)

def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di

In [3]:
def get_files(filepath,expression='*.json'):
    '''
    Walks over a directory and its children to get all children json files pathes
    Arguments:
    file_path: string that specifies the path to the data parent directory 
    Returns:
    all_files: List of all the filepaths of the matching expression files included in the directory
    '''
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,expression))
        for f in files :
            all_files.append(os.path.abspath(f))
    return all_files

In [4]:
cd ..\..

E:\CVprojects\Butterflies


In [5]:
data_path = 'Data2'

In [6]:
files_paths = get_files(data_path,'*.jpg')
len(files_paths)

104

In [7]:
files_paths[0][:-6]+'.jpg'

'E:\\CVprojects\\Butterflies\\Data2\\001.Atrophaneura_horishanus_male_upper\\0.jpg'

In [8]:
images_classes = [os.path.basename(path.split('.')[0]) for path in files_paths]
classes_samples = {}
classes_names = {}
for path in files_paths:
    class_num = int(os.path.basename(path.split('.')[0]))-1
    class_name = path.split('.')[1].split('/')[0]
    if class_num not in classes_samples:
        classes_samples[class_num] = []
    classes_samples[class_num].append(path)
    classes_names[class_num] = class_name

## Downsampling
Each class will be limited to 90 samples, rest will be in the slow testing dataset

In [9]:
val_rat = 0.5

In [10]:
train_paths = []
val_paths = []

np.random.seed(0)
for key in classes_samples:
    cur_paths = np.array(classes_samples[key])
    l = len(cur_paths)
    l2 = max(l,1)
    num_val = int(l2*val_rat)

    all_indeces = np.arange(l)
    np.random.shuffle(all_indeces)

    all_indeces_down = all_indeces

    val_indeces = all_indeces[:num_val]
    train_indeces = all_indeces[num_val:]

    train_paths.extend(cur_paths[train_indeces])
    val_paths.extend(cur_paths[val_indeces])


In [11]:
len(train_paths)+len(val_paths)

104

In [12]:
split_dict = {
    'train':train_paths,
    'val':val_paths,
}
split_dict_name = os.path.join('configs\\splits',"split_dict2.pkl")
save_dict(split_dict,split_dict_name)